# Code was taken from [Keras-Team](https://github.com/keras-team/keras/blob/master/examples/lstm_seq2seq.py) and applied to chinese to english

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np

In [4]:
batch_size = 64 # training batch size
epochs = 100 # no. of epochs
latent_dim = 256 # Latent dim of encoding space
num_samples = 10_000 # no of training samples
data_path = "cmn.txt" # data

## Vectorize data

In [18]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

In [19]:
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    target_text, input_text, _ = line.split('\t')
    # tab as start, new line as end
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [20]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

In [23]:
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [24]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 2575
Number of unique output tokens: 75
Max sequence length for inputs: 20
Max sequence length for outputs: 32


In [25]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

In [61]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(target_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [62]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    
    for t, char in enumerate(target_text):
        # offset by one, "teacher forcing"
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t>0:
            # decoder_target_data ahead of decoder_input_data by one timestamp
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
            
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

####  process input sequence

In [64]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# discard `encoder_outputs` keep states
encoder_states = [state_h, state_c]

#### setup the decoder, using `encoder_states` as initial state

In [66]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# setup decoder to return full output seqs,
# and return internal states as well. they do not use
# return states in training but use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                    initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

## Define the model

In [68]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

#### Train

In [69]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
             metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
         batch_size=batch_size,
         epochs=epochs,
         validation_split=0.2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 12s 2ms/sample - loss: 2.2591 - accuracy: 0.4285 - val_loss: 2.7207 - val_accuracy: 0.2671
Epoch 2/100
8000/8000 [==============================] - 4s 517us/sample - loss: 1.8186 - accuracy: 0.5012 - val_loss: 2.2440 - val_accuracy: 0.3684
Epoch 3/100
8000/8000 [==============================] - 4s 513us/sample - loss: 1.5442 - accuracy: 0.5617 - val_loss: 2.0344 - val_accuracy: 0.4112
Epoch 4/100
8000/8000 [==============================] - 4s 507us/sample - loss: 1.4158 - accuracy: 0.5872 - val_loss: 1.9722 - val_accuracy: 0.4233
Epoch 5/100
8000/8000 [==============================] - 4s 504us/sample - loss: 1.3348 - accuracy: 0.6070 - val_loss: 1.8815 - val_accuracy: 0.4450
Epoch 6/100
8000/8000 [==============================] - 4s 516us/sample - loss: 1.2660 - accuracy: 0.6260 - val_loss: 1.7863 - val_accuracy: 0.4685
Epoch 7/100
8000/8000 [==============================] - 4s

8000/8000 [==============================] - 4s 503us/sample - loss: 0.2725 - accuracy: 0.9209 - val_loss: 1.9304 - val_accuracy: 0.5816
Epoch 56/100
8000/8000 [==============================] - 4s 499us/sample - loss: 0.2647 - accuracy: 0.9233 - val_loss: 1.8733 - val_accuracy: 0.5899
Epoch 57/100
8000/8000 [==============================] - 4s 501us/sample - loss: 0.2549 - accuracy: 0.9260 - val_loss: 1.9097 - val_accuracy: 0.5871
Epoch 58/100
8000/8000 [==============================] - 4s 501us/sample - loss: 0.2475 - accuracy: 0.9276 - val_loss: 1.9679 - val_accuracy: 0.5811
Epoch 59/100
8000/8000 [==============================] - 4s 504us/sample - loss: 0.2391 - accuracy: 0.9305 - val_loss: 1.9591 - val_accuracy: 0.5867
Epoch 60/100
8000/8000 [==============================] - 4s 502us/sample - loss: 0.2311 - accuracy: 0.9336 - val_loss: 2.0154 - val_accuracy: 0.5799
Epoch 61/100
8000/8000 [==============================] - 4s 500us/sample - loss: 0.2223 - accuracy: 0.9362 - val

In [70]:
# Save model
model.save('s2s_zh2en.h5')

#### Next: inference mode (sampling).
#### Here's the drill:
#### 1) encode input and retrieve initial decoder state
#### 2) run one step of decoder with this initial state
#### and a "start of sequence" token as target.
#### Output will be the next target token
#### 3) Repeat with the current target token and current states

In [71]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [72]:
# reverse lookup to decode sequence
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [75]:
def decode_sequence(input_seq):
    # encode input as state vectors
    states_value = encoder_model.predict(input_seq)
    
    # generate empty target sequence of length 1
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # populate the first char of target sequence with start char.
    target_seq[0, 0, target_token_index['\t']] = 1.
    
    # sampling loop for batch sequences
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
        [target_seq] + states_value)
        
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char
        
        # exit condition: max length
        # or stop char
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True
        
        # update the target sequence (of length 1)/
        target_seq[0, 0, sampled_token_index] = 1.
        
        # update states
        states_value = [h, c]
        
    return decoded_sentence

In [77]:
for seq_index in range(5):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: 嗨。
Decoded sentence: Hi'lllvvvevseeeeeeeeeeeeevvvvvvvv
-
Input sentence: 你好。
Decoded sentence: Hew laaavairiiiiiiiiiiiiiiiiiiiid
-
Input sentence: 你用跑的。
Decoded sentence: Rungreesssttt sssasvieeeeeeeeeeee
-
Input sentence: 等等！
Decoded sentence: Waitcccey!,    'sssllllll

-
Input sentence: 你好。
Decoded sentence: Hew laaavairiiiiiiiiiiiiiiiiiiiid
